In [2]:
import numpy as np
from pathlib import Path
import skimage as sk
import cv2 as cv
import skimage.io as skio
import skimage.color as skcolor
# import scipy.signal as scsignal
import matplotlib.pyplot as plt

def load_image(path, greyscale=True):
    im = skio.imread(path)
    im = sk.img_as_float(im)
    
    # remove alpha if its there
    if im.shape[-1] == 4:
        im = skcolor.rgba2rgb(im)

    if greyscale and im.ndim == 3:
        im = skcolor.rgb2gray(im)
    return im

def save_image(path, im):
    skio.imsave(path, sk.img_as_ubyte(im))

In [4]:
# Get Points
from helper import get_points

im1 = load_image('./images/set1/L1.jpg', greyscale=False)
im2 = load_image('./images/set1/L2.jpg', greyscale=False)

%matplotlib qt
points1, points2 = get_points(im1, im2)
%matplotlib inline



Please select 10 points in each image for alignment.


In [5]:
def computeH(im1_pts, im2_pts):
    A = []
    b = []
    for i in range(len(im1_pts)):
        x1, y1 = im1_pts[i]
        x2, y2 = im2_pts[i]
        A.append([x1, y1, 1, 0, 0, 0, -x1*x2, -y1*x2])
        A.append([0, 0, 0, x1, y1, 1, -x1*y2, -y1*y2])

        b.append(x2)
        b.append(y2)
    
    A = np.array(A)
    b = np.array(b)
    h = np.linalg.lstsq(A, b, rcond=None)[0]
    H = np.append(h, 1).reshape(3, 3)
    return H

H = computeH(points1, points2)
print(H)



[[ 2.49090884e+00 -5.16690613e-02 -4.70612937e+03]
 [ 6.15450759e-01  1.99569550e+00 -1.42574315e+03]
 [ 3.78078622e-04 -1.99672303e-05  1.00000000e+00]]


In [17]:
import scipy.ndimage as ndi

def get_output_shape(im, H):
    h, w = im.shape[:2]

    corners = np.array([
        [0, 0, 1],
        [w, 0, 1],
        [w, h, 1],
        [0, h, 1]
    ]).T

    warped_corners = H @ corners
    warped_corners /= warped_corners[2,:]

    ref_corners = corners[:2, :]

    all_corners = np.hstack((warped_corners[:2,:], ref_corners))

    x_min, y_min = np.min(all_corners, axis=1)
    x_max, y_max = np.max(all_corners, axis=1)
    
    output_shape = (int(np.ceil(y_max - y_min)), int(np.ceil(x_max - x_min)))
    
    offset = np.array([
        [1, 0, -x_min],
        [0, 1, -y_min],
        [0, 0, 1]
    ])

    return output_shape, offset

def warpImageNearestNeighbor(im,H):
    H_inv = np.linalg.inv(H)
    output = np.zeros_like(im.shape[2:], dtype=im.dtype)
    h, w = im.shape[:2]

    for i in range(h):
        for j in range(w):
            p = np.array([j, i, 1])
            p_transformed = H_inv @ p
            p_transformed /= p_transformed[2]
            x, y = int(round(p_transformed[0])), int(round(p_transformed[1]))
            if 0 <= x < w and 0 <= y < h:
                output[i, j] = im[y, x]
    return output

def warpImageBilinear(im, H):
    output_shape, M_offset = get_output_shape(im, H)
    out_h, out_w = output_shape[:2]
    h, w = im.shape[:2]
    
    # have to build in shift to map it for cropping
    composite_H = M_offset @ H
    H_inv = np.linalg.inv(composite_H)
    
    y_out, x_out = np.meshgrid(np.arange(out_h), np.arange(out_w), indexing='ij')
    out_coords = np.vstack((x_out.ravel(), y_out.ravel(), np.ones(out_h * out_w)))
    src_coords_h = H_inv @ out_coords # inverse map output coordinates
    src_x = (src_coords_h[0,:] / src_coords_h[2,:]).reshape(output_shape)
    src_y = (src_coords_h[1,:] / src_coords_h[2,:]).reshape(output_shape) # normalize them with the w
    
    # top left and bottom right
    x0 = np.floor(src_x).astype(int)
    y0 = np.floor(src_y).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1
    
    # differences
    dx = src_x - x0
    dy = src_y - y0
    
    # limit to points only in region of source image
    valid_points = (x0 >= 0) & (x1 < w) & (y0 >= 0) & (y1 < h)
    
    output = np.zeros(output_shape + im.shape[2:], dtype=im.dtype)

    for channel in range(im.shape[2]):
        im_ch = im[:,:,channel]
        
        # get all of the point mappings
        top_left = im_ch[y0[valid_points], x0[valid_points]]
        top_right = im_ch[y0[valid_points], x1[valid_points]]
        bottom_left = im_ch[y1[valid_points], x0[valid_points]]
        bottom_right = im_ch[y1[valid_points], x1[valid_points]]

        dx_masked = dx[valid_points]
        dy_masked = dy[valid_points]
        
        top_interp = top_left * (1 - dx_masked) + top_right * dx_masked
        bottom_interp = bottom_left * (1 - dx_masked) + bottom_right * dx_masked
        interpolated_values = top_interp * (1 - dy_masked) + bottom_interp * dy_masked
        
        channel_out = np.zeros(output_shape, dtype=im.dtype)
        channel_out[valid_points] = interpolated_values
        output[:,:,channel] = channel_out
        
    return output

imwarped_nn = warpImageNearestNeighbor(im1,H)
imwarped_bil = warpImageBilinear(im1,H)

save_image('./images/set1/L1_warped.jpg', imwarped_nn)
save_image('./images/set1/L1_warped_bilinear.jpg', imwarped_bil)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [15]:
# Test Rectification

points_dst = np.array([
    [0, 0],
    [499, 0],
    [499, 499],
    [0, 499]
])

im_points = points1.copy()
im_points = im_points[:4]

#H = computeH(im_points, points_dst)
rectified = warpImageNearestNeighbor(im1,H)

save_image('./images/set1/L1_rectified.jpg', rectified)

